In [684]:
import geemap
import ee
import folium
import pandas as pd
import hvplot.pandas

# imports
import panel as pn
pn.extension('plotly')

In [685]:
ee.Initialize()

In [686]:
start_date = "2015-01-01"
end_date = "2020-01-01"

In [687]:
##dataset = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(start_date, end_date)).first()
dataset = ee.Image('USDA/NASS/CDL/2020').select('cropland')

In [688]:
##cropLandcover = dataset.select('cropland')
cultivated = dataset.select('cultivated')


##Cropland classes
crops = [
         {'Value': 1, 'Color': 'ffd300', 'Description': 'Corn'},
         {'Value': 5, 'Color': '267000', 'Description': 'Soybean'},
         {'Value': 22, 'Color': '896054', 'Description': 'Durum Wheat'},
         {'Value': 23, 'Color': 'd8b56b', 'Description': 'Spring Wheat'},
         {'Value': 24, 'Color': 'a57000', 'Description': 'Winter Wheat'}
        ]


## Convert list to a df
crops_df = pd.DataFrame(crops)

## Set index to Value
crops_df = crops_df.set_index('Value')



In [689]:
Map = geemap.Map()

##State boundaries
usState = ee.FeatureCollection("users/czhang11/boundary/cb_2017_us_state_5m")

Map.addLayer(usState.style(color= '6e6e6e', fillColor= '00000000', width=1), {}, 'U.S. ASD boundary');


In [690]:

vis_params = {}
crop_array = []
crop_color = []
def set_layers(Crop):
    
    crop_array = [crops_df.loc[crops_df.Description == Crop].index.item()]
    crop_color = [crops_df.loc[crops_df.Description == Crop].Color.item()]        
    
    vis_params = {
        'min': 1,
        'max': 1,
        'palette': crop_color
    }
    
    crop_cover = dataset.remap(crop_array, crop_array, None)
    
    
    Map.addLayer(crop_cover, vis_params, Crop + ' Landcover 2020')
    


In [691]:
from panel.interact import interact

interact(set_layers, Crop=crops_df['Description'])


Column
    [0] Column
        [0] Select(name='Crop', options=['Corn', 'Soybean', ...], value='Corn')
    [1] Row
        [0] Str(None, name='interactive36821')

In [692]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [693]:
import ipywidgets as widgets
from ipyleaflet import WidgetControl

In [694]:
# Add an output widget to the map
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

In [695]:
coordinates = []
def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        Map.default_style = {'cursor': 'wait'}
        coord = ee.Geometry.Point([latlon[1], latlon[0]])
        state_coord = usState.filterBounds(coord);
        coordinates = latlon
        #print(state_coord.getInfo())
        
        display(cropland_bystate(state_coord))
        
        with output_widget:
            clear_output()
            output_widget.clear_output()
            
            print(latlon) 
    Map.default_style = {'cursor': 'pointer'}
    
    
Map.on_interaction(handle_interaction)

In [696]:
Map

Map(bottom=1851.0, center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=H…

:NdOverlay   [Variable]
   :Curve   [Year]   (value)

In [697]:

def cropland_bystate(state_region):
    
    cropland_pixels_list_state = []
    cropland_pixels_dic_state = {}
    cropland_years_state = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
    
    for year in cropland_years_state:    

        for index, row in crops_df.iterrows():
            cropland_pixels_dic_state = {}
            cropland_pixels_dic_state['Year'] = year
            cropland_pixels_dic_state['Value'] = index
            cropland_pixels_dic_state['Color'] = row.Color
            cropland_pixels_dic_state['Description'] = row.Description
            cropland_pixels_dic_state['Pixel'] = pixel_retrieval(year, index, state_region, 300)

            cropland_pixels_list_state.append(cropland_pixels_dic_state)
        
    state_pixel_df = pd.DataFrame(cropland_pixels_list_state)
    
    state_pivoted_pixel_data = state_pixel_df.pivot(index='Year', columns='Description', values='Pixel')

    
    return state_pivoted_pixel_data.hvplot(label='Yearly Cropland Size Variance', xlabel='Year', ylabel='Land Size', height=400, width=1024)
    

In [698]:
region = usState


In [699]:
def pixel_retrieval(year, cropid, pixel_region, scale):
    
    google_image = ee.Image('USDA/NASS/CDL/' + str(year)).select('cropland').remap([cropid], [cropid], None)
    
    pixels = google_image.reduceRegion(reducer=ee.Reducer.count(), 
                                       geometry=pixel_region.geometry(),
                                       scale=scale,
                                       maxPixels=1e9)
    return pixels.getInfo()['remapped']

In [181]:
cropland_pixels_list = []
cropland_pixels_dic = {}

cropland_years = [2015, 2016, 2017, 2018, 2019, 2020]

for year in cropland_years:    
    
    for index, row in crops_df.iterrows():
        cropland_pixels_dic = {}
        cropland_pixels_dic['Year'] = year
        cropland_pixels_dic['Value'] = index
        cropland_pixels_dic['Color'] = row.Color
        cropland_pixels_dic['Description'] = row.Description
        cropland_pixels_dic['Pixel'] = pixel_retrieval(year, index, region, 1000)
        
        cropland_pixels_list.append(cropland_pixels_dic)
        
    
cropland_pixels_list

[{'Year': 2015,
  'Value': 1,
  'Color': 'ffd300',
  'Description': 'Corn',
  'Pixel': 384953},
 {'Year': 2015,
  'Value': 5,
  'Color': '267000',
  'Description': 'Soybean',
  'Pixel': 349530},
 {'Year': 2015,
  'Value': 22,
  'Color': '896054',
  'Description': 'Durum Wheat',
  'Pixel': 6902},
 {'Year': 2015,
  'Value': 23,
  'Color': 'd8b56b',
  'Description': 'Spring Wheat',
  'Pixel': 58753},
 {'Year': 2015,
  'Value': 24,
  'Color': 'a57000',
  'Description': 'Winter Wheat',
  'Pixel': 139921},
 {'Year': 2016,
  'Value': 1,
  'Color': 'ffd300',
  'Description': 'Corn',
  'Pixel': 416331},
 {'Year': 2016,
  'Value': 5,
  'Color': '267000',
  'Description': 'Soybean',
  'Pixel': 353569},
 {'Year': 2016,
  'Value': 22,
  'Color': '896054',
  'Description': 'Durum Wheat',
  'Pixel': 8319},
 {'Year': 2016,
  'Value': 23,
  'Color': 'd8b56b',
  'Description': 'Spring Wheat',
  'Pixel': 51019},
 {'Year': 2016,
  'Value': 24,
  'Color': 'a57000',
  'Description': 'Winter Wheat',
  'Pixel

In [183]:
pixel_df = pd.DataFrame(cropland_pixels_list)

In [388]:
pixel_df

,Year,Value,Color,Description,Pixel
0,2015,1,ffd300,Corn,384953
1,2015,5,267000,Soybean,349530
2,2015,22,896054,Durum Wheat,6902
3,2015,23,d8b56b,Spring Wheat,58753
4,2015,24,a57000,Winter Wheat,139921
5,2016,1,ffd300,Corn,416331
6,2016,5,267000,Soybean,353569
7,2016,22,896054,Durum Wheat,8319
8,2016,23,d8b56b,Spring Wheat,51019
9,2016,24,a57000,Winter Wheat,131507


In [309]:
##Pivot data
pivoted_pixel_data = pixel_df.pivot(index='Year', columns='Description', values='Pixel')

pivoted_pixel_data


#.opts(line_color='red')

Description,Corn,Durum Wheat,Soybean,Spring Wheat,Winter Wheat
Year,,,,,
2015,384953,6902,349530,58753,139921
2016,416331,8319,353569,51019,131507
2017,396572,8041,400015,52385,117622
2018,394153,7677,402132,61452,118010
2019,409970,4983,333823,62552,116411
2020,419050,6223,371763,62284,107772


In [219]:
pivoted_pixel_data.hvplot(label='Yearly Cropland Size Variance', xlabel='Year', ylabel='Land Size', height=400, width=1024)


:NdOverlay   [Variable]
   :Curve   [Year]   (value)

In [771]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

Output(layout=Layout(border='1px solid black'))

In [772]:
button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

display(button)
button.on_click(display_out)

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

In [773]:
#out = widgets.Output(layout={'border': '1px solid black'})
#display(out)

In [774]:
def display_out(b):
    with out:
        for i in range(5):
            print(i, 'Hello world!')

In [775]:
display_out(button) #simulates button press (calls the function by passing the button object)